### Learning a Latent Space of Multitrack Measures

MusicVAE learns a latent space of musical sequences. We apply the MusicVAE framework to single measures of multi-instrument General MIDI, a symbolic music representation that uses a standard set of 128 instrument sounds.

The models in this notebook are capable of encoding and decoding single measures of up to 8 tracks, optionally conditioned on an underlying chord. Encoding transforms a single measure into a vector in a latent space, and decoding transforms a latent vector back into a measure. Both encoding and decoding are performed hierarchically, with one level operating on tracks and another operating on the notes (and choice of instrument) in each track.

#### Env Setup

In [1]:
!gsutil -q -m cp gs://download.magenta.tensorflow.org/models/music_vae/multitrack/* /content/
!gsutil -q -m cp gs://download.magenta.tensorflow.org/soundfonts/SGM-v2.01-Sal-Guit-Bass-V1.3.sf2 /content/

!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev
!pip install -qU magenta pyfluidsynth pretty_midi

Selecting previously unselected package libfluidsynth1:amd64.
(Reading database ... 144487 files and directories currently installed.)
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 1.6MB 1.3MB/s 
     |████████████████████████████████| 5.6MB 7.3MB/s 
     |████████████████████████████████| 256kB 24.7MB/s 
     |████████████████████████████████| 215kB 20.8MB/s 
     |████████████████████████████████| 204kB 21.4MB/s 
     |████████████████████████████████| 2.3MB 23.9MB/s 
     |████████████████████████████████| 358kB 31.2MB/s 
     |████████████████████████████████| 1.5MB 28.8MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
     |████████████████████████████████| 71kB 7.

In [2]:
import ctypes.util
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return ctypes.util.find_library(lib)
ctypes.util.find_library = proxy_find_library

In [3]:
import os

import numpy as np
import magenta.music as mm
import tensorflow.compat.v1 as tf

from google.colab import files
from magenta.models.music_vae import configs
from magenta.music.sequences_lib import concatenate_sequences
from magenta.models.music_vae.trained_model import TrainedModel

In [4]:
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term
